### CNN
* http://andrew.gibiansky.com/blog/machine-learning/convolutional-neural-networks/
* http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/
* https://medium.com/@ageitgey/machine-learning-is-fun-part-3-deep-learning-and-convolutional-neural-networks-f40359318721
* https://algotravelling.com/ru/%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B5-%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5-%D1%8D%D1%82%D0%BE-%D0%B2%D0%B5%D1%81%D0%B5%D0%BB%D0%BE-3/
* https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
* https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b
* https://www.asozykin.ru/courses/nnpython
* https://www.tensorflow.org/tutorials/deep_cnn#convolutional-neural-networks

In [10]:
from scipy import ndimage
import numpy as np
import pandas as pd
import os
from IPython.display import display, Image
from scipy import misc
import matplotlib.pyplot as plt
import tensorflow
import seaborn

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

%matplotlib inline

### Keras

Виктор
* арима для каждого игрока
* последние 2 месяца
* фб профит либа

* уменьшить число фич - можно побить бейслайн

vgg для 224\224
keras vgg
уменьшить количество каналов в 4 раза
resnet: первый слов свертка 7х7 пулинг 3х3 убрать
конва + идентити блок




In [2]:
img_width = img_height = 32
img_channels = 3
initializer = 'orthogonal'

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer=initializer, 
           use_bias=False, input_shape=(img_width, img_height, img_channels)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
# model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer=initializer, 
#            use_bias=False, input_shape=(img_width, img_height, img_channels)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer=initializer, use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
# model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer=initializer, use_bias=False))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.2))

model.add(Conv2D(128, (1, 1), padding='same', kernel_initializer=initializer, use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
# model.add(Conv2D(128, (1, 1), padding='same', kernel_initializer=initializer, use_bias=False))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(256, kernel_initializer=initializer))
model.add(Activation('softmax'))

In [3]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

In [4]:
batch_size = 128

dir_transformed = '../output/transformed32'
dir_train = os.path.join(dir_transformed, 'train')
dir_val = os.path.join(dir_transformed, 'validation')
dir_test = os.path.join(dir_transformed, 'test')

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    dir_train,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1. / 255)

val_generator = val_datagen.flow_from_directory(
    dir_val,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = val_datagen.flow_from_directory(
    dir_test,
    target_size=(img_width, img_height),
    #class_mode=None,
    batch_size=batch_size,
    shuffle=False)

Found 16665 images belonging to 256 classes.
Found 5435 images belonging to 256 classes.
Found 7680 images belonging to 1 classes.


In [9]:
#early_stopping = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=0, mode='auto')
#model_checkpoint = ModelCheckpoint(filepath='model.hdf5', save_best_only=True, save_weights_only=False)

callback_1 = TensorBoard(log_dir='./logs/logs_{}'.format(NAME), histogram_freq=0, write_graph=False, write_images=False)
callback_2 = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, mode='auto')
callback_3 = ModelCheckpoint("models/model_{}.hdf5".format(NAME), monitor='val_acc', save_best_only=True, verbose=0)

NameError: name 'NAME' is not defined

In [5]:
%%time
model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=20,
    validation_data=val_generator,
    validation_steps=20
)

model.save('first_try.h5')

Epoch 1/20
 85/100 [========================>.....] - ETA: 270s - loss: 4.9985 - acc: 0.1036

KeyboardInterrupt: 

Todo:
model.fit(…, callbacks=[keras.callbacks.TensorBoard(log_dir='./logs')])
keras + TensorBoard? callback keras.callbacks.TensorBoard(log_dir=‘./logs’)

In [53]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 16, 16, 16)        1200      
_________________________________________________________________
batch_normalization_27 (Batc (None, 16, 16, 16)        48        
_________________________________________________________________
activation_36 (Activation)   (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 16, 16, 32)        4608      
_________________________________________________________________
batch_normalization_28 (Batc (None, 16, 16, 32)        96        
_________________________________________________________________
activation_37 (Activation)   (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 16, 16, 32)        0         
__________

In [54]:
results = model.predict_generator(test_generator, steps=300)
results = np.argmax(results, axis=1)+1

In [55]:
# create submission dataframe (in specified kaggle format)
sub = pd.DataFrame(
    data=[row for row in zip([i.split("/")[1] for i in test_generator.filenames], results.astype(int).tolist())],
    columns=['image', 'class']
)

In [56]:
sub.to_csv('keras_do-05.csv', encoding='utf-8', index=False)

In [ ]:
from keras.models import model_from_json
import h5py
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
import os, shutil
folder = 'data/keras/train/test'

files = ['11660.jpg', '12705.jpg', '13044.jpg', '14305.jpg', '14353.jpg', '14917.jpg',
 '16561.jpg', '18023.jpg', '18553.jpg', '18699.jpg', '18890.jpg', '19102.jpg',
 '2512.jpg', '25542.jpg', '25974.jpg', '2610.jpg', '2623.jpg', '26539.jpg',
 '27451.jpg', '28278.jpg', '28891.jpg', '29901.jpg', '31811.jpg', '3866.jpg',
 '5034.jpg', '5159.jpg', '5248.jpg', '5502.jpg', '5708.jpg', '7178.jpg']

for file in os.listdir(folder):
    if file in files:
        file_path = os.path.join(folder, file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            #elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)